In [4]:
%pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install numpy==1.23.0

Note: you may need to restart the kernel to use updated packages.Collecting numpy==1.23.0
  Using cached numpy-1.23.0-cp310-cp310-win_amd64.whl.metadata (2.2 kB)
Using cached numpy-1.23.0-cp310-cp310-win_amd64.whl (14.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aquacrop 3.0.9 requires numpy==1.22.4, but you have numpy 1.23.0 which is incompatible.


In [6]:
%pip install pandas==2.1.4 scipy==1.10.1 cffi==1.16.0 matplotlib==3.5.0

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install aquacrop

  Using cached numpy-1.22.4-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
Using cached numpy-1.22.4-cp310-cp310-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
os.environ['DEVELOPMENT'] = 'True'

In [11]:
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, FieldMngt, GroundWater
from aquacrop.utils import prepare_weather, get_filepath

import pandas as pd
import matplotlib.pyplot as plt


INFO:root:Running the simulation in development mode.


In [12]:
df = pd.read_csv('test_data.csv')
df


,지점명,Date,AvgTemp,MaxTemp,MinTemp,Precipitation,DayOfYear,ReferenceET
0,구미,2023-01-01,0.6,7.4,-5.9,0.0,1,2.426
1,구미,2023-01-02,-0.5,4.1,-4.4,0.0,2,1.829
2,구미,2023-01-03,-0.9,4.4,-5.3,0.0,3,1.916
3,구미,2023-01-04,-0.7,6.7,-6.0,0.0,4,2.227
4,구미,2023-01-05,-1.0,7.1,-7.0,0.0,5,2.315
...,...,...,...,...,...,...,...,...
360,구미,2023-12-27,2.4,9.0,-1.4,0.0,361,2.325
361,구미,2023-12-28,1.6,9.2,-3.7,0.0,362,2.492
362,구미,2023-12-29,1.8,8.8,-3.6,0.0,363,2.474
363,구미,2023-12-30,1.8,5.5,-3.2,0.3,364,2.077


In [13]:
import math
from datetime import datetime

In [14]:
def hargreaves_evapotranspiration(tmax, tmin, tmean, lat, doy):
    Gsc = 0.0820
    phi = math.radians(lat)
    delta = 0.409 * math.sin((2 * math.pi / 365) * doy - 1.39)
    omega_s = math.acos(-math.tan(phi) * math.tan(delta))
    Ra = (24 * 60 / math.pi) * Gsc * (omega_s * math.sin(phi) * math.sin(delta) + math.cos(phi) * math.cos(delta) * math.sin(omega_s))
    ET0 = 0.0023 * Ra * ((tmax - tmin) ** 0.5) * (tmean + 17.8)
    return ET0

In [15]:
def get_day_of_year(date_str):
    date = datetime.strptime(date_str, "%Y-%m-%d")
    return date.timetuple().tm_yday

In [16]:
latitude = 36.1190
df['DayOfYear'] = df['Date'].apply(get_day_of_year)  # DOY 계산
df['ReferenceET'] = round(df.apply(lambda row: hargreaves_evapotranspiration(row['MaxTemp'], row['MinTemp'], row['AvgTemp'], latitude, row['DayOfYear']), axis=1),3)

print(df)

df.to_csv('test_data.csv', index=False)

    지점명        Date  AvgTemp  MaxTemp  MinTemp  Precipitation  DayOfYear  \
0    구미  2023-01-01      0.6      7.4     -5.9            0.0          1   
1    구미  2023-01-02     -0.5      4.1     -4.4            0.0          2   
2    구미  2023-01-03     -0.9      4.4     -5.3            0.0          3   
3    구미  2023-01-04     -0.7      6.7     -6.0            0.0          4   
4    구미  2023-01-05     -1.0      7.1     -7.0            0.0          5   
..   ..         ...      ...      ...      ...            ...        ...   
360  구미  2023-12-27      2.4      9.0     -1.4            0.0        361   
361  구미  2023-12-28      1.6      9.2     -3.7            0.0        362   
362  구미  2023-12-29      1.8      8.8     -3.6            0.0        363   
363  구미  2023-12-30      1.8      5.5     -3.2            0.3        364   
364  구미  2023-12-31      4.0      7.8      1.9            0.7        365   

     ReferenceET  
0          2.426  
1          1.829  
2          1.916  
3          

In [17]:
# CSV 파일 읽기
filepath = 'C:\\Users\\SSAFY\\Desktop\\Data\\S11P21D207\\DT\\FastAPI\\todo\\irr\\test_data.csv'
weather_df = pd.read_csv(filepath)

# Date 열을 Day, Month, Year로 분리
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df['Day'] = weather_df['Date'].dt.day
weather_df['Month'] = weather_df['Date'].dt.month
weather_df['Year'] = weather_df['Date'].dt.year

# 필요 없는 열 삭제 (DayOfYear는 필요 없음)
weather_df = weather_df.drop(columns=['Date', 'DayOfYear'])

# prepare_weather 함수가 기대하는 열 순서로 재정렬
weather_df = weather_df[['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]

# 데이터 출력
print(weather_df)

# 필요한 경우 DataFrame을 prepare_weather 함수에 넘기기 전에 저장
# weather_df.to_csv('processed_test_data.csv', index=False)


     Day  Month  Year  MinTemp  MaxTemp  Precipitation  ReferenceET
0      1      1  2023     -5.9      7.4            0.0        2.426
1      2      1  2023     -4.4      4.1            0.0        1.829
2      3      1  2023     -5.3      4.4            0.0        1.916
3      4      1  2023     -6.0      6.7            0.0        2.227
4      5      1  2023     -7.0      7.1            0.0        2.315
..   ...    ...   ...      ...      ...            ...          ...
360   27     12  2023     -1.4      9.0            0.0        2.325
361   28     12  2023     -3.7      9.2            0.0        2.492
362   29     12  2023     -3.6      8.8            0.0        2.474
363   30     12  2023     -3.2      5.5            0.3        2.077
364   31     12  2023      1.9      7.8            0.7        1.908

[365 rows x 7 columns]


In [18]:
silt_loam = Soil(soil_type='SiltLoam')
maize = Crop('Maize', planting_date='05/01')

In [19]:
InitWC = InitialWaterContent(
    wc_type='Num',  # 값의 유형을 수치적으로 설정
    value=[0.3]  # 모든 토양층에서 0.3 m³/m³로 설정
)

In [20]:
# 'Day', 'Month', 'Year'를 하나의 'Date'로 결합
weather_df['Date'] = pd.to_datetime(weather_df[['Year', 'Month', 'Day']])

# 불필요한 'Day', 'Month', 'Year' 열 삭제
weather_df = weather_df.drop(columns=['Day', 'Month', 'Year'])

# 필요한 열로 재정렬
weather_df = weather_df[['Date', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]

# 결과 출력
print(weather_df.head())

# weather_df.to_csv('processed_test_data.csv', index=False)

        Date  MinTemp  MaxTemp  Precipitation  ReferenceET
0 2023-01-01     -5.9      7.4            0.0        2.426
1 2023-01-02     -4.4      4.1            0.0        1.829
2 2023-01-03     -5.3      4.4            0.0        1.916
3 2023-01-04     -6.0      6.7            0.0        2.227
4 2023-01-05     -7.0      7.1            0.0        2.315


In [21]:
weather_df = weather_df.dropna()

# sim_start_time과 sim_end_time을 문자열로 제공
model = AquaCropModel(
    sim_start_time='2023/05/01',
    sim_end_time='2023/10/30',
    weather_df=weather_df,
    soil=silt_loam,
    crop=maize,
    initial_water_content=InitWC
)


In [22]:
# weather_df의 데이터 타입 확인
print(weather_df.dtypes)

# MinTemp, MaxTemp 열을 float으로 강제 변환
weather_df['MinTemp'] = weather_df['MinTemp'].astype(float)
weather_df['MaxTemp'] = weather_df['MaxTemp'].astype(float)

Date             datetime64[ns]
MinTemp                 float64
MaxTemp                 float64
Precipitation           float64
ReferenceET             float64
dtype: object


In [27]:
print(weather_df.notna().all())
print(weather_df.columns.duplicated())

Date             True
MinTemp          True
MaxTemp          True
Precipitation    True
ReferenceET      True
dtype: bool
[False False False False False]


In [26]:
model.run_model(till_termination=True)


c:\Users\SSAFY\AppData\Local\Programs\Python\Python310\lib\site-packages\aquacrop\entities\soil.py:363: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.profile = self.profile.fillna(method="ffill")


TypeError: '<' not supported between instances of 'float' and 'Timestamp'